##### Copyright 2018 The TensorFlow Constrained Optimization Authors. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

> http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Problem Setup

This is a simple example of recall-constrained optimization on simulated data: we seek a classifier that minimizes the average hinge loss while constraining recall to be at least 90%.

We'll start with the required imports&mdash;notice the definition of `tfco`:

In [ ]:
import math
import numpy as np
from six.moves import xrange
import tensorflow as tf

# Use the GitHub version of TFCO
!pip install git+https://github.com/google-research/tensorflow_constrained_optimization
import tensorflow_constrained_optimization as tfco

We'll next create a simple simulated dataset by sampling 1000 random 10-dimensional feature vectors from a Gaussian, finding their labels using a random "ground truth" linear model, and then adding noise by randomly flipping 200 labels.

In [ ]:
# Create a simulated 10-dimensional training dataset consisting of 1000 labeled
# examples, of which 800 are labeled correctly and 200 are mislabeled.
num_examples = 1000
num_mislabeled_examples = 200
dimension = 10
# We will constrain the recall to be at least 90%.
recall_lower_bound = 0.9

# Create random "ground truth" parameters for a linear model.
ground_truth_weights = np.random.normal(size=dimension) / math.sqrt(dimension)
ground_truth_threshold = 0

# Generate a random set of features for each example.
features = np.random.normal(size=(num_examples, dimension)).astype(
    np.float32) / math.sqrt(dimension)
# Compute the labels from these features given the ground truth linear model.
labels = (np.matmul(features, ground_truth_weights) >
          ground_truth_threshold).astype(np.float32)
# Add noise by randomly flipping num_mislabeled_examples labels.
mislabeled_indices = np.random.choice(
    num_examples, num_mislabeled_examples, replace=False)
labels[mislabeled_indices] = 1 - labels[mislabeled_indices]

# Constant Tensors containing the labels and features.
constant_labels = tf.constant(labels, dtype=tf.float32)
constant_features = tf.constant(features, dtype=tf.float32)

We're now ready to construct our model, and the corresponding optimization problem. We'll use a linear model of the form $f(x) = w^T x - t$, where $w$ is the `weights`, and $t$ is the `threshold`.

In [ ]:
# Create variables containing the model parameters.
weights = tf.Variable(tf.zeros(dimension), dtype=tf.float32, name="weights")
threshold = tf.Variable(0.0, dtype=tf.float32, name="threshold")

# The predictions are a nullary function returning a Tensor to support eager
# mode. In graph mode, you can simply use:
#   predictions = (
#       tf.tensordot(constant_features, weights, axes=(1, 0)) - threshold)
def predictions():
  return tf.tensordot(constant_features, weights, axes=(1, 0)) - threshold

Now that we have the output of our linear model (in the `predictions` variable), we can move on to constructing the optimization problem. At this point, there are two ways to proceed:

1.  We can use the rate helpers provided by the TFCO library. This is the easiest way to construct optimization problems based on *rates* (where a "rate" is the proportion of training examples on which some event occurs).
2.  We could instead create an implementation of the [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py) interface. This is the most flexible approach. In particular, it is not limited to problems expressed in terms of rates.

We'll now consider each of these two options in turn.

### Option 1: rate helpers

The main motivation of TFCO is to make it easy to create and optimize constrained problems written in terms of linear combinations of *rates*, where a "rate" is the proportion of training examples on which an event occurs (e.g. the false positive rate, which is the number of negatively-labeled examples on which the model makes a positive prediction, divided by the number of negatively-labeled examples). Our current example (minimizing a hinge relaxation of the error rate subject to a recall constraint) is such a problem.

In [ ]:
# Like the predictions, in eager mode, the labels should be a nullary function
# returning a Tensor. In graph mode, you can drop the lambda.
context = tfco.rate_context(predictions, labels=lambda: constant_labels)
problem = tfco.RateMinimizationProblem(
    tfco.error_rate(context), [tfco.recall(context) >= recall_lower_bound])

The first argument of all rate-construction helpers ([`error_rate`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/binary_rates.py) and [`recall`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/binary_rates.py) are the ones used here) is a "context" object, which represents what we're taking the rate *of*. For example, in a fairness problem, we might wish to constrain the [`positive_prediction_rate`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/binary_rates.py)s of two protected classes (i.e. two subsets of the data) to be similar. In that case, we would create a context representing the entire dataset, then call the context's `subset` method to create contexts for the two protected classes, and finally call the [`positive_prediction_rate`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/binary_rates.py) helper on the two resulting contexts. We're not trying to do anything so advanced here, however, so we only create a single context, representing the entire dataset.

In addition to the context, rate-construction helpers also take two optional named parameters&mdash;not used here&mdash;named `penalty_loss` and `constraint_loss`, of which the former is used to define the proxy constraints, and the latter the "true" constraints. These default to the hinge and zero-one losses, respectively. The consequence of this is that we will attempt to minimize the average hinge loss (a relaxation of the error rate using the `penalty_loss`), while constraining the *true* recall (using the `constraint_loss`) by essentially learning how much we should penalize the hinge-constrained recall (`penalty_loss`, again).

The [`RateMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/rate_minimization_problem.py) class implements the [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py) interface, and is constructed from a rate expression to be minimized (the first parameter), subject to a list of rate constraints (the second). Using this class is typically more convenient and readable than constructing a [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py) manually: the objects returned by `error_rate` and `recall`&mdash;and all other rate-constructing and rate-combining functions&mdash;can be manipulated using python arithmetic operators (e.g. "`0.5 * tfco.error_rate(context1) - tfco.true_positive_rate(context2)`"), or converted into a constraint using a comparison operator.

### Option 2: explicit [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py)

For problems that cannot be easily expressed using the rate helpers, one could instead an explicit implementation of the [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py) interface. The current task (minimizing the average hinge loss subject to a recall constraint) is a rate-based problem, but for illustrative reasons we will show how to create a [`ConstrainedMinimizationProblem`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/constrained_minimization_problem.py) for this task.

The constructor takes three parameters: a `Tensor` containing the classification labels (0 or 1) for every training example, another `Tensor` containing the model's predictions on every training example (sometimes called the "logits"), and the lower bound on recall that will be enforced using a constraint.

As before, this implementation will contain both constraints *and* proxy constraints: the former represents the constraint that the true recall (defined in terms of the *number* of true positives) be at least `recall_lower_bound`, while the latter represents the same constraint, but on a hinge approximation of the recall.

In [ ]:
class ExampleProblem(tfco.ConstrainedMinimizationProblem):

  def __init__(self, labels, predictions, recall_lower_bound):
    self._labels = labels
    self._predictions = predictions
    self._recall_lower_bound = recall_lower_bound
    # The number of positively-labeled examples.
    self._positive_count = tf.reduce_sum(self._labels)

  @property
  def num_constraints(self):
    return 1

  def objective(self):
    # In eager mode, the predictions must be a nullary function returning a
    # Tensor. In graph mode, they could be either such a function, or a Tensor
    # itself.
    predictions = self._predictions
    if callable(predictions):
      predictions = predictions()
    return tf.compat.v1.losses.hinge_loss(labels=self._labels,
                                          logits=predictions)

  def constraints(self):
    # In eager mode, the predictions must be a nullary function returning a
    # Tensor. In graph mode, they could be either such a function, or a Tensor
    # itself.
    predictions = self._predictions
    if callable(predictions):
      predictions = predictions()
    # Recall that the labels are binary (0 or 1).
    true_positives = self._labels * tf.cast(predictions > 0, dtype=tf.float32)
    true_positive_count = tf.reduce_sum(true_positives)
    recall = true_positive_count / self._positive_count
    # The constraint is (recall >= self._recall_lower_bound), which we convert
    # to (self._recall_lower_bound - recall <= 0) because
    # ConstrainedMinimizationProblems must always provide their constraints in
    # the form (tensor <= 0).
    #
    # The result of this function should be a tensor, with each element being
    # a quantity that is constrained to be non-positive. We only have one
    # constraint, so we return a one-element tensor.
    return self._recall_lower_bound - recall

  def proxy_constraints(self):
    # In eager mode, the predictions must be a nullary function returning a
    # Tensor. In graph mode, they could be either such a function, or a Tensor
    # itself.
    predictions = self._predictions
    if callable(predictions):
      predictions = predictions()
    # Use 1 - hinge since we're SUBTRACTING recall in the constraint function,
    # and we want the proxy constraint function to be convex. Recall that the
    # labels are binary (0 or 1).
    true_positives = self._labels * tf.minimum(1.0, predictions)
    true_positive_count = tf.reduce_sum(true_positives)
    recall = true_positive_count / self._positive_count
    # Please see the corresponding comment in the constraints property.
    return self._recall_lower_bound - recall

problem = ExampleProblem(
    labels=constant_labels,
    predictions=predictions,
    recall_lower_bound=recall_lower_bound,
)

## Wrapping up

We're almost ready to construct and train our model, but first we'll create a couple of functions to measure its performance. We're interested in two quantities: the average hinge loss (which we seek to minimize), and the recall (which we constrain).

In [ ]:
def average_hinge_loss(labels, predictions):
  # Recall that the labels are binary (0 or 1).
  signed_labels = (labels * 2) - 1
  return np.mean(np.maximum(0.0, 1.0 - signed_labels * predictions))

def recall(labels, predictions):
  # Recall that the labels are binary (0 or 1).
  positive_count = np.sum(labels)
  true_positives = labels * (predictions > 0)
  true_positive_count = np.sum(true_positives)
  return true_positive_count / positive_count

As was mentioned in [README.md](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/README.md), a Lagrangian optimizer ([lagrangian_optimizer.py](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/train/lagrangian_optimizer.py)) often suffices for problems without proxy constraints, but a proxy-Lagrangian optimizer ([proxy_lagrangian_optimizer.py](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/train/proxy_lagrangian_optimizer.py)) is recommended for problems *with* proxy constraints. Since this problem contains proxy constraints, we use a proxy-Lagrangian optimizer.

For this problem, the constraint is fairly easy to satisfy, so we can use the same "inner" optimizer (an `AdagradOptimizer` with a learning rate of 1) for optimization of both the model parameters (`weights` and `threshold`), and the internal parameters associated with the constraints (these are the analogues of the Lagrange multipliers used by the proxy-Lagrangian optimizer). For more difficult problems, it will often be necessary to use different optimizers, with different learning rates (presumably found via a hyperparameter search): to accomplish this, pass *both* the `optimizer` and `constraint_optimizer` parameters to `ProxyLagrangianOptimizerV1`'s (or `ProxyLagrangianOptimizerV2`'s) constructor.

Since this is a convex problem (both the objective and proxy constraint functions are convex), we can just take the last iterate. Periodic snapshotting, and the use of the [`find_best_candidate_distribution`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/candidates.py) or [`find_best_candidate_index`](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/candidates.py) functions, is generally only necessary for non-convex problems (and even then, it isn't *always* necessary).

In [ ]:
if tf.executing_eagerly():
  # In eager mode, we use a V2 optimizer (a tf.keras.optimizers.Optimizer). A V1
  # optimizer, however, would work equally well.
  optimizer = tfco.ProxyLagrangianOptimizerV2(
      optimizer=tf.keras.optimizers.Adagrad(learning_rate=1.0),
      num_constraints=problem.num_constraints)
  # In addition to the model parameters (weights and threshold), we also need to
  # optimize over any trainable variables associated with the problem (e.g.
  # implicit slack variables and weight denominators), and those associated with
  # the optimizer (the analogues of the Lagrange multipliers used by the
  # proxy-Lagrangian formulation).
  var_list = ([weights, threshold] + problem.trainable_variables +
              optimizer.trainable_variables())

  for ii in xrange(1000):
    optimizer.minimize(problem, var_list=var_list)

  trained_weights = weights.numpy()
  trained_threshold = threshold.numpy()

else:  # We're in graph mode.
  # In graph mode, we use a V1 optimizer (a tf.compat.v1.train.Optimizer). A V2
  # optimizer, however, would work equally well.
  optimizer = tfco.ProxyLagrangianOptimizerV1(
      optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=1.0))
  train_op = optimizer.minimize(problem)

  with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for ii in xrange(1000):
      session.run(train_op)

    trained_weights, trained_threshold = session.run((weights, threshold))

trained_predictions = np.matmul(features, trained_weights) - trained_threshold
print("Constrained average hinge loss = %f" % average_hinge_loss(
    labels, trained_predictions))
print("Constrained recall = %f" % recall(labels, trained_predictions))

Constrained average hinge loss = 0.722943
Constrained recall = 0.900826


As we hoped, the recall is extremely close to 90%&mdash;and, thanks to the fact that the optimizer uses a (hinge) proxy constraint only when needed, and the actual (zero-one) constraint whenever possible, this is the *true* recall, not a hinge approximation.

### Unconstrained Model

For comparison, let's try optimizing the same problem *without* the recall constraint:

In [ ]:
if tf.executing_eagerly():
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=1.0)
  var_list = [weights, threshold]

  for ii in xrange(1000):
    # For optimizing the unconstrained problem, we just minimize the
    # "objective" portion of the minimization problem.
    optimizer.minimize(problem.objective, var_list=var_list)

  trained_weights = weights.numpy()
  trained_threshold = threshold.numpy()

else:  # We're in graph mode.
  optimizer = tf.compat.v1.train.AdagradOptimizer(learning_rate=1.0)
  # For optimizing the unconstrained problem, we just minimize the "objective"
  # portion of the minimization problem.
  train_op = optimizer.minimize(problem.objective())

  with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for ii in xrange(1000):
      session.run(train_op)

    trained_weights, trained_threshold = session.run((weights, threshold))

trained_predictions = np.matmul(features, trained_weights) - trained_threshold
print("Unconstrained average hinge loss = %f" % average_hinge_loss(
    labels, trained_predictions))
print("Unconstrained recall = %f" % recall(labels, trained_predictions))

Unconstrained average hinge loss = 0.615532
Unconstrained recall = 0.762397


Because there is no constraint, the unconstrained problem does a better job of minimizing the average hinge loss, but naturally doesn't approach 90% recall.